# Creating Data Set

### Loading libraries

In [1]:
import pandas as pd

In [2]:
import re

### Loading separate data sets

In [3]:
# Loading datasets 
lyrics_df = pd.read_csv("labeled_lyrics_cleaned.csv")  

genders_df = pd.read_csv("Gender_Dateset_2.5.csv")

# Verify
lyrics_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'labeled_lyrics_cleaned.csv'

In [ ]:
genders_df.head()

### Initial data wrangling

In [ ]:
# Removing columns
lyrics_df = lyrics_df.drop(columns=["Unnamed: 0", "song", "label"])

In [ ]:
genders_df = genders_df[["NAME", "GENDER", "IS_BAND"]]

In [4]:
lyrics_df.columns

NameError: name 'lyrics_df' is not defined

In [9]:
genders_df.columns

Index(['NAME', 'GENDER', 'IS_BAND'], dtype='object')

In [10]:
lyrics_df.head()

,artist,seq
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r..."
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m..."
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\..."
4,Elijah Blake,"I see a midnight panther, so gallant and so br..."


In [11]:
genders_df.head()

,NAME,GENDER,IS_BAND
0,Anthony Moschella,NaN,NaN
1,thelostnoiz,NaN,NaN
2,Meech Lucidreamz,NaN,False
3,Johnny Mo Gospel,NaN,False
4,Oneiro,NaN,NaN


In [12]:
# So we know what to do with bands
# Bands have many genders linked, can discuss later
print(genders_df[genders_df["IS_BAND"] == True]["GENDER"].value_counts())



male                     2198
mixed                     889
female                    323
non-binary                108
not specified              59
androgynous                18
transgender                12
gender non-conforming       7
gender neutral              7
transgender woman           5
genderqueer                 5
genderfluid                 3
agender                     2
transgender man             1
two-spirit                  1
trans-masculine             1
Name: GENDER, dtype: int64


In [13]:
# Check if name has missing values or is not string and remove those rows
lyrics_df = lyrics_df.dropna(subset=["artist"])
gender_df = genders_df.dropna(subset=["NAME"])

lyrics_df = lyrics_df[lyrics_df["artist"].apply(lambda x: isinstance(x, str))]
genders_df = genders_df[genders_df["NAME"].apply(lambda x: isinstance(x, str))]

print("Done")


Done


### Cleaning names

In [14]:
# Function to clean name
def clean_name(name):
    name = name.lower().strip()  
    name = re.sub(r'[^\w\s]', '', name)   # Got re from chatgpt
    
    return name

In [15]:
# Cleaning both artists names
lyrics_df["artist_clean"] = lyrics_df["artist"].apply(clean_name)
genders_df["artist_clean"] = genders_df["NAME"].apply(clean_name)

In [16]:
genders_df.head()

,NAME,GENDER,IS_BAND,artist_clean
0,Anthony Moschella,NaN,NaN,anthony moschella
1,thelostnoiz,NaN,NaN,thelostnoiz
2,Meech Lucidreamz,NaN,False,meech lucidreamz
3,Johnny Mo Gospel,NaN,False,johnny mo gospel
4,Oneiro,NaN,NaN,oneiro


### Keep only unique values

In [17]:
# Remove dupes in genders
duplicated_artists = genders_df[genders_df.duplicated(subset='artist_clean', keep=False)]

# Check count
print(len(duplicated_artists))

149745


In [18]:
# List duplicates
duplicated_artists_names = genders_df[genders_df.duplicated(subset='artist_clean', keep=False)]['artist_clean'].unique()

# Remove relevant rows
genders_df_cleaned = genders_df[~genders_df['artist_clean'].isin(duplicated_artists_names)]

# Check remaining rows
print(len(genders_df_cleaned))

949666


### Merge

In [19]:
# Merge, keep all rows from lyrics_df
merged_df = lyrics_df.merge(genders_df_cleaned, left_on="artist_clean", right_on="artist_clean", how="inner")

# Check the number of rows in the merged dataset
print(len(merged_df))


124242


In [20]:
merged_df.head(5)

,artist,seq,artist_clean,NAME,GENDER,IS_BAND
0,Elijah Harris,To believe\r\nOr not to believe\r\nThat is the...,elijah harris,Elijah Harris,NaN,False
1,Elis Regina,"Deve ser bom, deve ser bom\r\nTo fechado pra b...",elis regina,Elis Regina,NaN,False
2,Elis Regina,Once there was a way to get back homeward\r\nO...,elis regina,Elis Regina,NaN,False
3,Elis Regina,Gracias a la vida que me ha dado tanto\r\nMe d...,elis regina,Elis Regina,NaN,False
4,Elis Regina,Sad is to live in solitude\r\nFar from your tr...,elis regina,Elis Regina,NaN,False


### Filter for gender

In [21]:
# Remove rows where the GENDER column is NaN
merged_df = merged_df.dropna(subset=["GENDER"])

print(len(merged_df))


21057


In [22]:
# Drop duplicate artists names
columns_to_drop = ["artist_clean", "NAME"]

merged_df = merged_df.drop(columns=columns_to_drop)

In [23]:
merged_df.head()

,artist,seq,GENDER,IS_BAND
60,Elizabeth Naccarato,"Oh, Danny boy, the pipes, the pipes are callin...",female,False
98,Ella Fitzgerald,I never feel a thing is real\r\nWhen I'm away ...,female,False
99,Ella Fitzgerald,"I really can't stay\r\nBut, baby, it's cold ou...",female,False
100,Ella Fitzgerald,All my life\r\nI've been waiting for you\r\nMy...,female,False
101,Ella Fitzgerald,I'll be down to get you in a taxi honey\r\nBet...,female,False


### Export 

In [25]:
merged_df.to_csv("cleaned_lyrics_gender.csv", index=False)
print("done")

done
